In [ ]:
import genetic_algorithm.population as population
import genetic_algorithm.transforms as transforms
import genetic_algorithm.optimize as optimize
import genetic_algorithm.utility as utility
import pandas as pd
import numpy as np
import genetic_algorithm.visualization as visualization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import genetic_algorithm.evaluation as evaluation


data = pd.read_csv("../data/ES15.csv")
x_raw = data.values

dirpath = utility.fetch_new_run_dirpath()

pop = population.generate_random_forest(250, 12)

np.seterr(all='ignore')
best_forest, best_scores, best_overtime = optimize.optimize_constants(
    pop, x_raw, sthresh_q=.1, run_dir=dirpath
)

img = visualization.visualize_tree(best_forest[best_scores.index(min(best_scores))], run_dir=dirpath)
newforest , newscores = population.extract_n_best_trees(best_forest, best_scores, -1, run_dir=dirpath)

x_ = transforms.forest2features(
    population=newforest,
    x_raw=x_raw
)

ynew = np.roll(x_raw[:, 3], shift=-1)
y_ = np.log(ynew / x_raw[:, 3])

X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size=0.3, shuffle=False)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model, history = evaluation.standard_NN_construction(X_train, y_train)
evaluation.standard_NN_evaluation(X_train, X_test, y_train, y_test, model, history, dirpath)

In [ ]:
import genetic_algorithm.utility as utility

utility.demo_constopt_nn()